# Query elasticity data from Materials Project

<div style="
    border-left: 4px solid #5bc0de;
    background-color: #f4f8fb;
    color: #000000;
    padding: 10px;
    margin: 10px 0;
">
<strong>📝 Note</strong><br><br>
<strong>Remove entries</strong> having a formation energy (or energy above the convex hull) <strong>more than 150meV</strong> and those having <strong>negative G_Voigt, G_Reuss, G_VRH, K_Voigt, K_Reuss, or K_VRH</strong> and those <strong>failing G_Reuss <= G_VRH <= G_Voigt |  K_Reuss <= K_VRH <= K_Voigt</strong>
</div>


In [ ]:
import pandas as pd
import numpy as np
from mp_api.client import MPRester

In [ ]:
mpr = MPRester(mute_progress_bars=True, api_key="add_your_mp_api_key_here") # See https://next-gen.materialsproject.org/api

In [ ]:
# Exclude Nobel gases Kr, Ne, Xe
mpids_nobel_gases = ["mp-612118", "mp-111", "mp-570510"]

In [ ]:
# load the mpids for which lobster calcs are available in our LOBSTER database. Data for particular mpids will be queried
with open("lobster_calc_mpids.txt", "r", encoding="utf-8") as f:
    mpids = [line.rstrip("\n") for line in f]

In [ ]:
# Remove nobel gases
for mpid in mpids_nobel_gases:
    mpids.remove(mpid)

In [ ]:
df_targets = pd.DataFrame(index=list(mpids), columns=["structure"])

**Note that the DB version used was `2025.09.25` when queried**

In [ ]:
mpr.get_database_version() 

In [ ]:
chunk_size = 1000  # Define the size of each chunk

for i in range(0, len(mpids), chunk_size):
    mpids_chunk = mpids[i:i+chunk_size]
    data_chunk_elast = mpr.materials.elasticity.search(list(mpids_chunk), g_voigt=(0, np.inf),
                     g_reuss=(0, np.inf),
                     g_vrh=(0, np.inf),
                     k_reuss=(0, np.inf),
                     k_voigt=(0, np.inf),
                     k_vrh=(0, np.inf),
                     poisson_ratio=(0, np.inf),
                     elastic_anisotropy=(0, np.inf), )
    data_chunk_summary = mpr.materials.summary.search(material_ids=list(mpids_chunk))

    for data in data_chunk_summary:
        df_targets.at[data.material_id.string, "formula_pretty"] = data.formula_pretty
        df_targets.at[data.material_id.string, "structure"] = data.structure
        df_targets.at[data.material_id.string, "energy_above_hull_meV"] = data.energy_above_hull * 1E-6
        df_targets.at[data.material_id.string, "is_metal"] = data.is_metal
        df_targets.at[data.material_id.string, "is_deprecated"] = data.deprecated
        df_targets.at[data.material_id.string, "is_stable"] = data.is_stable

    for data in data_chunk_elast:
        df_targets.at[data.material_id.string, "k_voigt"] = data.bulk_modulus.voigt
        df_targets.at[data.material_id.string, "k_reuss"] = data.bulk_modulus.reuss
        df_targets.at[data.material_id.string, "k_vrh"] = data.bulk_modulus.vrh
        df_targets.at[data.material_id.string, "g_voigt"] = data.shear_modulus.voigt
        df_targets.at[data.material_id.string, "g_reuss"] = data.shear_modulus.reuss
        df_targets.at[data.material_id.string, "g_vrh"] = data.shear_modulus.vrh

In [ ]:
# Filter the data based on condition described at the top and leave out entires labelled as metals and deprecated
df_filtered_targets_semi = df_targets.loc[
    ((df_targets.k_voigt >= df_targets.k_vrh) & (df_targets.k_vrh >= df_targets.k_reuss) |
    (df_targets.g_voigt >= df_targets.g_vrh) & (df_targets.g_vrh >= df_targets.g_reuss))
    & (df_targets.is_metal==False) & 
(df_targets.energy_above_hull_meV <=150) & (df_targets.is_deprecated ==False)]

In [ ]:
df_k_vrh = df_filtered_targets_semi.loc[:, ["k_vrh"]]

df_k_vrh["log_k_vrh"] = np.log10(df_k_vrh["k_vrh"])
df_k_vrh.drop(columns=["k_vrh"], inplace=True)

# df_k_vrh.to_json("log_k_vrh.json") # Uncomment to save data

In [ ]:
df_g_vrh = df_filtered_targets_semi.loc[:, ["g_vrh"]]

df_g_vrh["log_g_vrh"] = np.log10(df_g_vrh["g_vrh"])
df_g_vrh.drop(columns=["g_vrh"], inplace=True)

# df_g_vrh.to_json("log_g_vrh.json") # Uncomment to save data